In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cvxpy as cp

## Task 2

### 2.1

In [ ]:
### Load electricity prices ###
price_path = os.path.join(os.getcwd(),'ElspotpricesEA.csv')
df_prices = pd.read_csv(price_path)

### Convert to datetime ###
df_prices["HourDK"] = pd.to_datetime(df_prices["HourDK"])
df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
df_prices['HourUTC'] = df_prices['HourUTC'].dt.tz_localize('UTC')
df_prices['HourDK'] = df_prices['HourUTC'].dt.tz_convert('CET')

### Filter only DK2 prices, restricting price list to DK2 ###
df_prices = df_prices.loc[df_prices['PriceArea']=="DK2"]

### Keep only the local time and price columns ###
df_prices = df_prices[['HourDK','SpotPriceDKK']]

### Reset the index ###
df_prices = df_prices.reset_index(drop=True)

### Adding columns for year, month, day, hour as datetime objects ###

df_prices["Year"]=df_prices["HourDK"].dt.year
df_prices["Month"]=df_prices["HourDK"].dt.month
df_prices["Day"]=df_prices["HourDK"].dt.day_of_year
df_prices["Hour"]=df_prices["HourDK"].dt.hour

### Filtering by year ###

df_prices_2019 = df_prices.loc[df_prices["Year"].isin([2019])]
df_prices_2020 = df_prices.loc[df_prices["Year"].isin([2020])]
df_prices_2021 = df_prices.loc[df_prices["Year"].isin([2021])]
df_prices_2022 = df_prices.loc[df_prices["Year"].isin([2022])]
df_prices_2023 = df_prices.loc[df_prices["Year"].isin([2023])]

### Finding mean prices for each day of the year ###

df_2019_365mean = df_prices_2019.groupby([df_prices["Day"]])["SpotPriceDKK"].mean().reset_index()
df_2020_365mean = df_prices_2020.groupby([df_prices["Day"]])["SpotPriceDKK"].mean().reset_index()
df_2021_365mean = df_prices_2021.groupby([df_prices["Day"]])["SpotPriceDKK"].mean().reset_index()
df_2022_365mean = df_prices_2022.groupby([df_prices["Day"]])["SpotPriceDKK"].mean().reset_index()
df_2023_365mean = df_prices_2023.groupby([df_prices["Day"]])["SpotPriceDKK"].mean().reset_index()

In [ ]:
# Parameters
Pmax = 5
energy_capacity = 10
n_c = 0.95
n_d = 0.95
Cmax = 1*energy_capacity
Cmin = 0.1*energy_capacity
C_0 = 0.5*energy_capacity
C_n = 0.5*energy_capacity

# Define the prices array
spot_prices2019 = df_2019_365mean["SpotPriceDKK"].values
spot_prices2020 = df_2020_365mean["SpotPriceDKK"].values #leap year
spot_prices2021 = df_2021_365mean["SpotPriceDKK"].values
spot_prices2022 = df_2022_365mean["SpotPriceDKK"].values
spot_prices2023 = df_2023_365mean["SpotPriceDKK"].values

In [ ]:
# Optimization Variables
n_nonleapyear = 365 # Number of steps
n_leapyear = 366 # Number of steps

# Non Leap Year
p_c_nonleap = cp.Variable(n_nonleapyear)
p_d_nonleap = cp.Variable(n_nonleapyear)
X_nonleap   = cp.Variable(n_nonleapyear)

# Leap Year
p_c_leap = cp.Variable(n_leapyear)
p_d_leap = cp.Variable(n_leapyear)
X_leap   = cp.Variable(n_leapyear)

In [ ]:
# Objective function for each year

cost2019 = cp.sum(-p_d_nonleap@spot_prices2019 + p_c_nonleap@spot_prices2019)
cost2020 = cp.sum(-p_d_leap   @spot_prices2020 + p_c_leap   @spot_prices2020)
cost2021 = cp.sum(-p_d_nonleap@spot_prices2021 + p_c_nonleap@spot_prices2021)
cost2022 = cp.sum(-p_d_nonleap@spot_prices2022 + p_c_nonleap@spot_prices2022)
cost2023 = cp.sum(-p_d_nonleap@spot_prices2023 + p_c_nonleap@spot_prices2023)

In [ ]:
# Constraints Leap Year
constraintsleap = [p_c_leap >= 0,
                   p_d_leap >= 0,
                   p_c_leap <= Pmax,
                   p_d_leap <= Pmax]
constraintsleap += [X_leap >= Cmin, X_leap <= Cmax]
constraintsleap += [X_leap[0]==C_0 + p_c_leap[0]*n_c - p_d_leap[0]/n_d]

for j in range(1,n_leapyear):
    constraintsleap += [X_leap[j]==X_leap[j-1] + p_c_leap[j]*n_c - p_d_leap[j]/n_d]

constraintsleap += [X_leap[n_leapyear-1]>=C_n]

# Constraints Non Leap Year
constraintsnonleap = [p_c_nonleap >= 0,
                      p_d_nonleap >= 0,
                      p_c_nonleap <= Pmax,
                      p_d_nonleap <= Pmax]
constraintsnonleap += [X_nonleap >= Cmin, X_nonleap <= Cmax]
constraintsnonleap += [X_nonleap[0]==C_0 + p_c_nonleap[0]*n_c - p_d_nonleap[0]/n_d]

for j in range(1,n_nonleapyear):
    constraintsnonleap += [X_nonleap[j]==X_nonleap[j-1] + p_c_nonleap[j]*n_c - p_d_nonleap[j]/n_d]

constraintsnonleap += [X_nonleap[n_nonleapyear-1]>=C_n]

In [ ]:
#Problem for each year 

###2019
problem2019 = cp.Problem(cp.Minimize(cost2019), constraintsnonleap)
problem2019.solve(solver=cp.CLARABEL)

print("\n--- Optimization Results 2019 ---")
#print(f"Charging power (p_c): {np.round(p_c_nonleap.value, 1)}")
#print(f"Discharging power (p_d): {np.round(p_d_nonleap.value, 1)}")
#print(f"State of charge (X): {np.round(X_nonleap.value, 1)}")
print(f"Total cost: {np.round(cost2019.value, 1)}")
print("-----------------------------")

###2020
problem2020 = cp.Problem(cp.Minimize(cost2020), constraintsleap)
problem2020.solve(solver=cp.CLARABEL)

print("\n--- Optimization Results 2020 ---")
#print(f"Charging power (p_c): {np.round(p_c_leap.value, 1)}")
#print(f"Discharging power (p_d): {np.round(p_d_leap.value, 1)}")
#print(f"State of charge (X): {np.round(X_leap.value, 1)}")
print(f"Total cost: {np.round(cost2020.value, 1)}")
print("-----------------------------")

###2021
problem2021 = cp.Problem(cp.Minimize(cost2021), constraintsnonleap)
problem2021.solve(solver=cp.CLARABEL)

print("\n--- Optimization Results 2021 ---")
#print(f"Charging power (p_c): {np.round(p_c_nonleap.value, 1)}")
#print(f"Discharging power (p_d): {np.round(p_d_nonleap.value, 1)}")
#print(f"State of charge (X): {np.round(X_nonleap.value, 1)}")
print(f"Total cost: {np.round(cost2021.value, 1)}")
print("-----------------------------")

###2022
problem2022 = cp.Problem(cp.Minimize(cost2022), constraintsnonleap)
problem2022.solve(solver=cp.CLARABEL)

print("\n--- Optimization Results 2022 ---")
#print(f"Charging power (p_c): {np.round(p_c_nonleap.value, 1)}")
#print(f"Discharging power (p_d): {np.round(p_d_nonleap.value, 1)}")
#print(f"State of charge (X): {np.round(X_nonleap.value, 1)}")
print(f"Total cost: {np.round(cost2022.value, 1)}")
print("-----------------------------")

###2023
problem2023 = cp.Problem(cp.Minimize(cost2023), constraintsnonleap)
problem2023.solve(solver=cp.CLARABEL)

print("\n--- Optimization Results 2023 ---")
#print(f"Charging power (p_c): {np.round(p_c_nonleap.value, 1)}")
#print(f"Discharging power (p_d): {np.round(p_d_nonleap.value, 1)}")
#print(f"State of charge (X): {np.round(X_nonleap.value, 1)}")
print(f"Total cost: {np.round(cost2023.value, 1)}")
print("-----------------------------")

In [ ]:
#Results for 2019

# Create the figure and axes objects for the two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4)  # Adjust space between plots

   # Plot the prices in the top subplot (exclude the first and last hours)
ax1.stairs(spot_prices2019, range(len(spot_prices2019) + 1), label='Prices', baseline = None, color='darkblue', linewidth=2)
ax1.set_xlabel('Hour', fontsize=12)
ax1.set_ylabel('Price [DKK/MWh]', fontsize=12)
ax1.set_title("Spot Prices Over 2019", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Plot the power in the bottom subplot (exclude the first and last hours)
ax2.stairs(p_c_nonleap.value, range(len(spot_prices2019) + 1), label='Charging Power', baseline = None, color='green', linewidth=2)
ax2.stairs(-p_d_nonleap.value, range(len(spot_prices2019) + 1), label='Discharging Power', baseline = None, color='red', linewidth=2)
ax2.set_xlabel('Hour', fontsize=12)
ax2.set_ylabel('Power [kW]', fontsize=12)
ax2.set_title("Battery Charging/Discharging Schedule", fontsize=14, fontweight='bold')
ax2.legend(loc='upper center', fontsize=10, frameon=True, shadow=True, ncol=2)
ax2.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Results for 2020

# Create the figure and axes objects for the two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4)  # Adjust space between plots

   # Plot the prices in the top subplot (exclude the first and last hours)
ax1.stairs(spot_prices2020, range(len(spot_prices2020) + 1), label='Prices', baseline = None, color='darkblue', linewidth=2)
ax1.set_xlabel('Hour', fontsize=12)
ax1.set_ylabel('Price [DKK/MWh]', fontsize=12)
ax1.set_title("Spot Prices Over 2020", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Plot the power in the bottom subplot (exclude the first and last hours)
ax2.stairs(p_c_leap.value, range(len(spot_prices2020) + 1), label='Charging Power', baseline = None, color='green', linewidth=2)
ax2.stairs(-p_d_leap.value, range(len(spot_prices2020) + 1), label='Discharging Power', baseline = None, color='red', linewidth=2)
ax2.set_xlabel('Hour', fontsize=12)
ax2.set_ylabel('Power [kW]', fontsize=12)
ax2.set_title("Battery Charging/Discharging Schedule", fontsize=14, fontweight='bold')
ax2.legend(loc='upper center', fontsize=10, frameon=True, shadow=True, ncol=2)
ax2.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Results for 2021

# Create the figure and axes objects for the two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4)  # Adjust space between plots

   # Plot the prices in the top subplot (exclude the first and last hours)
ax1.stairs(spot_prices2021, range(len(spot_prices2021) + 1), label='Prices', baseline = None, color='darkblue', linewidth=2)
ax1.set_xlabel('Hour', fontsize=12)
ax1.set_ylabel('Price [DKK/MWh]', fontsize=12)
ax1.set_title("Spot Prices Over 2021", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Plot the power in the bottom subplot (exclude the first and last hours)
ax2.stairs(p_c_nonleap.value, range(len(spot_prices2021) + 1), label='Charging Power', baseline = None, color='green', linewidth=2)
ax2.stairs(-p_d_nonleap.value, range(len(spot_prices2021) + 1), label='Discharging Power', baseline = None, color='red', linewidth=2)
ax2.set_xlabel('Hour', fontsize=12)
ax2.set_ylabel('Power [kW]', fontsize=12)
ax2.set_title("Battery Charging/Discharging Schedule", fontsize=14, fontweight='bold')
ax2.legend(loc='upper center', fontsize=10, frameon=True, shadow=True, ncol=2)
ax2.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Results for 2022

# Create the figure and axes objects for the two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4)  # Adjust space between plots

   # Plot the prices in the top subplot (exclude the first and last hours)
ax1.stairs(spot_prices2022, range(len(spot_prices2022) + 1), label='Prices', baseline = None, color='darkblue', linewidth=2)
ax1.set_xlabel('Hour', fontsize=12)
ax1.set_ylabel('Price [DKK/MWh]', fontsize=12)
ax1.set_title("Spot Prices Over 2022", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Plot the power in the bottom subplot (exclude the first and last hours)
ax2.stairs(p_c_nonleap.value, range(len(spot_prices2022) + 1), label='Charging Power', baseline = None, color='green', linewidth=2)
ax2.stairs(-p_d_nonleap.value, range(len(spot_prices2022) + 1), label='Discharging Power', baseline = None, color='red', linewidth=2)
ax2.set_xlabel('Hour', fontsize=12)
ax2.set_ylabel('Power [kW]', fontsize=12)
ax2.set_title("Battery Charging/Discharging Schedule", fontsize=14, fontweight='bold')
ax2.legend(loc='upper center', fontsize=10, frameon=True, shadow=True, ncol=2)
ax2.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Results for 2023

# Create the figure and axes objects for the two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
fig.subplots_adjust(hspace=0.4)  # Adjust space between plots

   # Plot the prices in the top subplot (exclude the first and last hours)
ax1.stairs(spot_prices2023, range(len(spot_prices2023) + 1), label='Prices', baseline = None, color='darkblue', linewidth=2)
ax1.set_xlabel('Hour', fontsize=12)
ax1.set_ylabel('Price [DKK/MWh]', fontsize=12)
ax1.set_title("Spot Prices Over 2023", fontsize=14, fontweight='bold')
ax1.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Plot the power in the bottom subplot (exclude the first and last hours)
ax2.stairs(p_c_nonleap.value, range(len(spot_prices2023) + 1), label='Charging Power', baseline = None, color='green', linewidth=2)
ax2.stairs(-p_d_nonleap.value, range(len(spot_prices2023) + 1), label='Discharging Power', baseline = None, color='red', linewidth=2)
ax2.set_xlabel('Hour', fontsize=12)
ax2.set_ylabel('Power [kW]', fontsize=12)
ax2.set_title("Battery Charging/Discharging Schedule", fontsize=14, fontweight='bold')
ax2.legend(loc='upper center', fontsize=10, frameon=True, shadow=True, ncol=2)
ax2.grid(True, linestyle=':', linewidth=0.7, alpha=0.8)

    # Show the plot
plt.tight_layout()
plt.show()

1. Which year is more profitable?
2. Can you provide a possible explanation?
3. What do you expect to happen to profits if you optimize for each year in one problem, instead of optimizing each day separately?



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example Data (Replace with your actual data)
x_values = range(len(spot_prices2019) + 1)  # Common x-axis values
years = [2019, 2020, 2021, 2022, 2023]  # Years corresponding to each dataset

# Simulated y-values (Replace with your actual y-values)
y_values = {
    2019: np.append(spot_prices2019, spot_prices2019[-1]) + np.random.normal(0, 0.1, len(x_values)),
    2020: spot_prices2020 + 0.5 + np.random.normal(0, 0.1, len(x_values)),
    2021: np.append(spot_prices2021, spot_prices2021[-1]) + 1.0 + np.random.normal(0, 0.1, len(x_values)),
    2022: np.append(spot_prices2022, spot_prices2022[-1]) + 1.5 + np.random.normal(0, 0.1, len(x_values)),
    2023: np.append(spot_prices2023, spot_prices2023[-1]) + 2.0 + np.random.normal(0, 0.1, len(x_values)),
}

# Define colors for each year
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Create plot
plt.figure(figsize=(10, 6))

for i, (year, y_vals) in enumerate(y_values.items()):
    plt.plot(x_values, y_vals, label=f'Year {year}', color=colors[i], linewidth=1)
    plt.fill_between(x_values, y_vals, alpha=0.5, color=colors[i])  # Fill area under curve

# Set y-axis ticks in intervals of 500
y_min, y_max = plt.ylim()  # Get the current y-axis limits
plt.xticks(np.arange(0, len(x_values), 15))
plt.yticks(np.arange(0, y_max, 500))  # Set ticks at intervals of 500

# Formatting
plt.xlabel('Day of the Year')
plt.ylabel('Price [DKK/MWh]')
plt.title('Average Daily Spot Price Fluctuations Over 5 Years')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()
